In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


#!/usr/bin/env python
# coding: utf-8

# In[ ]:


#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# get_ipython().system('pip install sentence_transformers')
import scipy.spatial
import numpy as np
import os, json
import glob
import re
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from tokenizers import Tokenizer
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)
import pandas as pd
import torch
import random
import itertools
import pickle
import time

start = time.time()

F1Measure_list = []
Recall_list = []
Accuracy_list = []
Precision_list = []
Hamming_Loss_list = []


#Load sentences & embeddings from disc
with open('/home/ubuntu/storage_data_new/df_claim_test_1M_pre_duplicates_removed_663.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_patent_test_embeddings_id = stored_data['patent_id']
    test_embeddings = stored_data['claim_embeddings']

with open('/home/ubuntu/storage_data_new/df_claim_train_1M_pre_duplicates_removed_663.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_patent_train_embeddings_id = stored_data['patent_id']
    claim_embeddings = stored_data['claim_embeddings']

# test_embeddings = torch.load('/home/ubuntu/storage_data/df_claim_test_133_n.pkl', map_location=torch.device('cpu'))
# claim_embeddings = torch.load('/home/ubuntu/storage_data/df_claim_train_133_n.pkl', map_location=torch.device('cpu'))


def get_top_n_similar_patents_df(new_claim, claim_embeddings):
    search_hits_list = []
    search_hits = util.semantic_search(new_claim, claim_embeddings, 10000, 5000000, 20)
#     # embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')
#     embedder = SentenceTransformer('/home/ubuntu/deeppatentsimilarity/results/stsb_augsbert_SS_roberta-base-2021-01-06_22-14-54')
# #     embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')
# #     embedder = SentenceTransformer('/home/ubuntu/deeppatentsimilarity/results/stsb_augsbert_SS_roberta-base-2021-01-06_22-14-54/')
# #     query_embeddings = embedder.encode([new_claim])
#     query_embeddings = new_claim
    
# #     query_embeddings = torch.load('/home/ubuntu/deeppatentsimilarity/patentdata/claim_embeddings_24_AugSBERT_test.pt', map_location=torch.device('cpu'))

# #     query_embeddings = tokenizer([new_claim], padding=True, truncation=True, max_length=128, return_tensors='pt')

#     # list of patent claims
# #     claim_embeddings = embedder.encode(claims)
# #     claim_embeddings = torch.load('/home/ubuntu/deeppatentsimilarity/patentdata/claim_embeddings_24_AugSBERT.pt', map_location=torch.device('cpu'))


#     # get top 100 patent claims based on cosine similarity
#     top_n = 40
#     distances = scipy.spatial.distance.cdist(query_embeddings, claim_embeddings, "cosine")[0]
# #     distances = sentence_transformers.util.semantic_search(query_embeddings, claim_embeddings, "cosine")[0]

#     results = zip(range(len(distances)), distances)
#     results = sorted(results, key=lambda x: x[1])

    # save similar patents info
    top_claim_order = []
    top_claim_ids = []
    top_similarity_scores = []
    
#     # print('New_claim: ' + new_claim + '\n')

#     # Find the closest 100 patent claims for each new_claim based on cosine similarity
#     for idx, distance in results[0:top_n]:
#         top_claim_ids.append(patent_id[idx])
#         top_claims.append(claims[idx])
#         top_similarity_scores.append(round((1-distance), 4))
# #         print('Patent ID: ' + str(patent_id[idx]))
# #         print('PubMed Claim: ' + claims[idx])
# #         print('Similarity Score: ' + "%.4f" % (1-distance))
# #         print('\n')
        
    for item in range(len(search_hits[0])):
        top_claim_order = search_hits[0][item].get('corpus_id')
        top_claim_ids.append(stored_patent_train_embeddings_id[top_claim_order])
        top_similarity_scores.append(search_hits[0][item].get('score'))
        
    top_100_similar_patents_df = pd.DataFrame({
        'top_claim_ids': top_claim_ids,
        'cosine_similarity': top_similarity_scores,
#         'claims': top_claims,
    })

        
    
    return top_100_similar_patents_df

def F1Measure(y_true, y_pred):
    save_F1 = []
    temp = 0
    for i in range(y_true.shape[0]):
        if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):
            continue
        temp_save = (2*sum(np.logical_and(y_true[i], y_pred[i])))/ (sum(y_true[i])+sum(y_pred[i]))
        save_F1.append(temp_save)
        temp += temp_save

    save_F1 = pd.DataFrame(save_F1)
    save_F1_ids = pd.concat([result, save_F1], axis=1, ignore_index=True)
#     save_F1_ids.to_csv(r'/home/ubuntu/storage_data/output/output_save_F1'+'-'+str(k)+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")+'.csv', index = False)
    return temp/ y_true.shape[0]

def Recall(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_pred[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_pred[i])
    return temp/ y_true.shape[0]

def Precision(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_true[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_true[i])
    return temp/ y_true.shape[0]

def Hamming_Loss(y_true, y_pred):
    temp=0
    for i in range(y_true.shape[0]):
        temp += np.size(y_true[i] == y_pred[i]) - np.count_nonzero(y_true[i] == y_pred[i])
    return temp/(y_true.shape[0] * y_true.shape[1])

def Accuracy(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / y_true.shape[0]

df_claim_cpc_test = pd.read_csv('/home/ubuntu/storage_data_new/df_claim_test_1M_pre_duplicates_removed_663.csv', encoding='ISO-8859-1')
df_claim_cpc_train = pd.read_csv('/home/ubuntu/storage_data_new/df_claim_train_1M_pre_duplicates_removed_663.csv', encoding='ISO-8859-1')
# df_claim_cpc_train_1000 = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/df_claim_cpc_all_len_150_200_1000.csv', encoding='ISO-8859-1')
# df_claim_cpc_test = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/prelabel/NewTest/df_1_L_43259_test_100.csv', encoding='ISO-8859-1')
# df_claim_cpc_train = pd.read_csv('/hsxome/ubuntu/deeppatentsimilarity/patentdata/prelabel/NewTest/df_1_L_43259.csv', encoding='ISO-8859-1')

claims = list(df_claim_cpc_train.text)
patent_id = list(df_claim_cpc_train.id)

listofpredictdfs = []

start = time.time()

for i in range(len(df_claim_cpc_test[:100])):
    get_top_n_similar_patents_df_predict = get_top_n_similar_patents_df(np.array(test_embeddings[i]).reshape(1,-1), claim_embeddings)
    result = pd.merge(get_top_n_similar_patents_df_predict, df_claim_cpc_train, left_on='top_claim_ids',right_on='id',how='left',suffixes=('_left','_right'))
    locals()["predict_n"+str(i)] = result.copy()
    listofpredictdfs.append("predict_n"+str(i))

# merged = list(itertools.chain(*listofpredictdfs))
# df_names = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/df_names.csv', encoding='ISO-8859-1')
# df_names = df_names.values.tolist()
# df_names..to_list()
 
# df = pd.concat(listofpredictdfs,keys= listofpredictdfs ,axis=0).reset_index(level=listofpredictdfs)
df = pd.concat(map(lambda x: eval(x), listofpredictdfs),keys= listofpredictdfs ,axis=0)

top_k = 20
for k in range(top_k):
    top_n = k
    predict = pd.DataFrame(columns= df_claim_cpc_test.columns[6:])
    for item in range(len(listofpredictdfs)):
        k_similar_patents = df.xs(listofpredictdfs[item]).nlargest(top_n, ['cosine_similarity'])
        result_k_similar_patents = pd.DataFrame(0, index=np.arange(1),columns= k_similar_patents.columns[8:])
        for i in range(top_n):
            result_k_similar_patents  = result_k_similar_patents + k_similar_patents.iloc[i, 8:].values
    #     result_k_similar_patents_r = result_k_similar_patents.reshape(1,600)
        result_k_similar_patents_df = pd.DataFrame(result_k_similar_patents, columns= k_similar_patents.columns[8:])
#     patent_id_input_res = pd.concat([input_patrnt_id_df, result_k_similar_patents], ignore_index=True)
        result_k_similar_patents_df.insert(0, "input_aptent_id", df_claim_cpc_test.id.iloc[item], True)
        locals()["predict"+str(item)] = result_k_similar_patents_df.copy()
        predict = pd.concat([predict, locals()["predict"+str(item)]], ignore_index=True)
        result_k_similar_patents_df = result_k_similar_patents_df[0:0]
# predict.to_csv(r'/home/ubuntu/deeppatentsimilarity/output/new_predict_result.csv'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S"), index = False)
# creating tensor from targets_df
#     predict = pd.read_csv('/home/ubuntu/deeppatentsimilarity/output/Multilabel17000.csv', encoding='ISO-8859-1')
    data = torch.tensor((predict.to_numpy()).astype(float), dtype=torch.float32)

# printing out result
# print(torch_tensor)
    m = nn.Sigmoid()
    
# input = torch.randn(2)
    output = m(data)
    output = (output>0.9).float()
    output_df = pd.DataFrame(output, columns=predict.columns).astype(float)
    
    y_pred = output_df.iloc[:, :-1].to_numpy()
    y_true = df_claim_cpc_test.iloc[:100, 6:].to_numpy()
    result = pd.concat([output_df, df_claim_cpc_test], axis=1, ignore_index=True)
#     result.to_csv(r'/home/ubuntu/storage_data/output/new_predict_result'+str(k)+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")+'.csv', index = False)


    
    
    F1Measure_list.append(F1Measure(y_true,y_pred))
    Recall_list.append(Recall(y_true,y_pred))
    Accuracy_list.append(Accuracy(y_true, y_pred))
    Precision_list.append(Precision(y_true,y_pred))
    Hamming_Loss_list.append(Hamming_Loss(y_true, y_pred))
    
    end = time.time()
    print(f"Runtime of the program is {end - start}")
    print("F1Measure: ", F1Measure_list[top_n])
    print("Recall: ", Recall_list[top_n])
    print("Accuracy: ", Accuracy_list[top_n])
    print("Precision: ", Precision_list[top_n])
    print("Hamming_Loss: ", Hamming_Loss_list[top_n])
    
output_d_metrics = {'F1Measure':F1Measure_list,'Recall_list':Recall_list, 'Accuracy_list':Accuracy_list,'Precision_list':Precision_list,'Hamming_Loss_list':Hamming_Loss_list}
output_df_metrics = pd.DataFrame(output_d_metrics)
# output_df_metrics.to_csv(r'/home/ubuntu/storage_data_new/output/output_df_metrics'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")+'.csv', index = False)

Runtime of the program is 62.681822299957275
F1Measure:  0.0
Recall:  0.0
Accuracy:  0.0
Precision:  0.0
Hamming_Loss:  0.002870090634441088
Runtime of the program is 66.3840844631195
F1Measure:  0.0
Recall:  0.0
Accuracy:  0.0
Precision:  0.0
Hamming_Loss:  0.002870090634441088
Runtime of the program is 71.70731830596924
F1Measure:  0.0
Recall:  0.0
Accuracy:  0.0
Precision:  0.0
Hamming_Loss:  0.002870090634441088
Runtime of the program is 78.95103883743286
F1Measure:  0.4888888888888887
Recall:  0.5716666666666667
Accuracy:  0.44041666666666673
Precision:  0.45233333333333337
Hamming_Loss:  0.001948640483383686
Runtime of the program is 88.63805508613586
F1Measure:  0.5997936507936505
Recall:  0.6741666666666666
Accuracy:  0.5353690476190477
Precision:  0.5822857142857144
Hamming_Loss:  0.0018126888217522659
Runtime of the program is 99.94861054420471
F1Measure:  0.6331587301587299
Recall:  0.6936666666666668
Accuracy:  0.5622857142857144
Precision:  0.6426190476190476
Hamming_Loss: